## Check Scene Viewing Eyetracking Files From All Blocks (.sh)

This combines two scripts and allows you to run checks on all 7 blocks

Checks recent SceneViewing Eyetracking files from all blocks for correct size and structure:  

- If Encoding: `1 edf && 2 dat && 1 log`
- If Retrieval: `1 edf && 1 dat && 2 log`

  Outputs into terminal

VERSION: `Check_Recent_SceneViewing_All_Blocks`

In [ ]:
#!/bin/bash

new_folders_found=false

check_encoding_block() {
    local session_folder=$1
    local folder_size=$(du -sh "$session_folder" | cut -f1)

    dat_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.dat" | wc -l)
    edf_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.edf" | wc -l)
    log_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.log" | wc -l)

    if [[ ! ($dat_count -eq 2 && $edf_count -eq 1 && $log_count -eq 1) ]]; then
        echo "  [UNEXPECTED FORMAT] Encoding block $session_folder [Size: $folder_size]. Contents:"
        ls -l "$session_folder"
    else
        echo "  [EXPECTED FORMAT] Encoding block $session_folder [Size: $folder_size]"
    fi
}

check_retrieval_block() {
    local session_folder=$1
    local folder_size=$(du -sh "$session_folder" | cut -f1)

    dat_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.dat" | wc -l)
    edf_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.edf" | wc -l)
    log_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.log" | wc -l)
    txt_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.txt" | wc -l)

    if [[ ! ($dat_count -eq 1 && $edf_count -eq 1 && $log_count -eq 2) || ($txt_count -gt 1) ]]; then
        echo "  [UNEXPECTED FORMAT] Retrieval block $session_folder. Contents:"
        ls -l "$session_folder"
    else
        echo "  [EXPECTED FORMAT] Retrieval block $session_folder [Size: $folder_size]"
    fi
}

for block_folder in */; do
    if [[ -d "$block_folder" ]]; then
        for id_folder in "${block_folder}"*/; do
            if [[ -d "$id_folder" ]]; then
                for session_folder in "${id_folder}"*/; do
                    if [[ -d "$session_folder" ]] && find "$session_folder" -maxdepth 0 -mtime -1 | read; then
                        new_folders_found=true
                        echo "Checking new or modified session folder: $session_folder"

                        if [[ $block_folder == Encoding* ]]; then
                            check_encoding_block "$session_folder"
                        elif [[ $block_folder == Retrieval* ]]; then
                            check_retrieval_block "$session_folder"
                        fi

                        echo "_________________________________________________________________"
                    fi
                done
            fi
        done
    fi
done

if ! $new_folders_found; then
    echo "No new or modified files in last 24 hours."
fi


Checks all SceneViewing Eyetracking files from all blocks for correct size and structure:  

- If Encoding: `1 edf && 2 dat && 1 log`
- If Retrieval: `1 edf && 1 dat && 2 log`


VERSION: `Check_All_SceneViewing_All_Blocks`

In [ ]:
#!/bin/bash

unexpected_found=false
output_file="unexpected_folders_report.txt"

> "$output_file"

check_encoding_block() {
    local session_folder=$1
    echo "Checking Encoding block: $session_folder"
    dat_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.dat" | wc -l)
    edf_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.edf" | wc -l)
    log_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.log" | wc -l)

    if [[ ! ($dat_count -eq 2 && $edf_count -eq 1 && $log_count -eq 1) ]]; then
        echo "[UNEXPECTED] Unexpected file structure in Encoding block $session_folder. Contents:" >> "$output_file"
        ls -l "$session_folder" >> "$output_file"
        echo "_________________________________________________________________" >> "$output_file"
        unexpected_found=true
    fi
}

check_retrieval_block() {
    local session_folder=$1
    echo "Checking Retrieval block: $session_folder"
    dat_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.dat" | wc -l)
    edf_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.edf" | wc -l)
    log_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.log" | wc -l)
    txt_count=$(find "$session_folder" -maxdepth 1 -type f -name "*.txt" | wc -l)

    if [[ ! ($dat_count -eq 1 && $edf_count -eq 1 && $log_count -eq 2) || ($txt_count -gt 1) ]]; then
        echo "[UNEXPECTED] Unexpected file structure in Retrieval block $session_folder. Contents:" >> "$output_file"
        ls -l "$session_folder" >> "$output_file"
        echo "_________________________________________________________________" >> "$output_file"
        unexpected_found=true
    fi
}

for block_folder in */; do
    if [[ -d "$block_folder" ]]; then
        for id_folder in "${block_folder}"*/; do
            if [[ -d "$id_folder" ]]; then
                for session_folder in "${id_folder}"*/; do
                    if [[ -d "$session_folder" ]]; then
                        if [[ $block_folder == Encoding* ]]; then
                            check_encoding_block "$session_folder"
                        elif [[ $block_folder == Retrieval* ]]; then
                            check_retrieval_block "$session_folder"
                        fi
                    fi
                done
            fi
        done
    fi
done

if ! $unexpected_found; then
    echo "All folders match expected structure." >> "$output_file"
else
    echo "There are folders with unexpected structure. See $output_file for details."
fi
